In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from math import atan2, degrees
from collections import defaultdict

def parse_pdb(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    return lines

def get_atom_coordinates(lines, residue_type, residue_number, atom_name, chain=None):
    for line in lines:
        if line.startswith('ATOM') or line.startswith('HETATM'):
            atom_residue_type = line[17:20].strip()
            atom_residue_number = int(line[22:26].strip())
            atom_chain = line[21].strip()  # Chain identifier
            atom_name_field = line[12:16].strip()

            if chain and atom_chain != chain:
                continue

            if (atom_residue_type == residue_type and atom_residue_number == residue_number 
                and atom_name_field == atom_name):
                try:
                    x = float(line[30:38].strip())
                    y = float(line[38:46].strip())
                    z = float(line[46:54].strip())
                    return (x, y, z)
                except ValueError:
                    print(f"Error parsing coordinates for {residue_type}{residue_number} {atom_name}")
                    return None
    print(f"Could not find {atom_name} atom for {residue_type} {residue_number}")
    return None

def vector(p1, p2):
    return np.array([p2[i] - p1[i] for i in range(3)])

def calculate_dihedral(p1, p2, p3, p4):
    """Calculate dihedral angle in degrees between 4 points"""
    b1 = vector(p1, p2)
    b2 = vector(p2, p3)
    b3 = vector(p3, p4)
    
    # Normalize b2 to avoid affecting magnitude of cross products
    b2 /= np.linalg.norm(b2)
    
    # Orthogonal vectors to the planes formed by b1-b2 and b2-b3
    n1 = np.cross(b1, b2)
    n2 = np.cross(b2, b3)
    
    # Normalize normal vectors
    n1 /= np.linalg.norm(n1)
    n2 /= np.linalg.norm(n2)
    
    # Angle between the normal vectors
    angle = atan2(np.dot(np.cross(n1, n2), b2), np.dot(n1, n2))
    
    return degrees(angle)

def ramachandran_plot(pdb_file, chain=None):
    lines = parse_pdb(pdb_file)
    
    # Dictionary to store φ and ψ angles for each residue
    phi_psi_angles = []
    residues = []
    
    prev_c = None
    for i, line in enumerate(lines):
        if line.startswith('ATOM') and line[12:16].strip() == 'N':
            res_number = int(line[22:26].strip())
            res_type = line[17:20].strip()

            n_coords = get_atom_coordinates(lines, res_type, res_number, 'N', chain)
            ca_coords = get_atom_coordinates(lines, res_type, res_number, 'CA', chain)
            c_coords = get_atom_coordinates(lines, res_type, res_number, 'C', chain)

            if prev_c and n_coords and ca_coords and c_coords:
                # Calculate phi
                phi = calculate_dihedral(prev_c, n_coords, ca_coords, c_coords)
            else:
                phi = None
            
            # Check if we can calculate psi for the current residue
            next_n_coords = get_atom_coordinates(lines, res_type, res_number + 1, 'N', chain)
            if ca_coords and c_coords and next_n_coords:
                psi = calculate_dihedral(n_coords, ca_coords, c_coords, next_n_coords)
            else:
                psi = None
            
            if phi is not None and psi is not None:
                phi_psi_angles.append((phi, psi))
                residues.append(res_number)

            prev_c = c_coords  # Save C atom of this residue for the next φ calculation

    return phi_psi_angles, residues

def classify_outliers(phi_psi_angles):
    """Classify outliers based on empirical Ramachandran regions"""
    allowed_region = lambda phi, psi: (-180 < phi < -30 and -180 < psi < 180)  # Placeholder rule
    outliers = [i for i, (phi, psi) in enumerate(phi_psi_angles) if not allowed_region(phi, psi)]
    return outliers

def plot_ramachandran(phi_psi_angles, outliers, residues):
    phi_angles = [phi for phi, psi in phi_psi_angles]
    psi_angles = [psi for phi, psi in phi_psi_angles]
    
    plt.figure(figsize=(6, 6))
    plt.xlim(-180, 180)
    plt.ylim(-180, 180)
    plt.xlabel('Phi (φ)')
    plt.ylabel('Psi (ψ)')
    plt.title('Ramachandran Plot')
    
    plt.scatter(phi_angles, psi_angles, color='blue', label='Allowed')
    
    # Highlight outliers
    outlier_angles = [phi_psi_angles[i] for i in outliers]
    outlier_residues = [residues[i] for i in outliers]
    plt.scatter([phi for phi, psi in outlier_angles], 
                [psi for phi, psi in outlier_angles], color='red', label='Outliers')

    plt.legend()
    plt.show()

    return outlier_residues


In [ ]:

# Main function
def main(pdb_file, chain=None):
    phi_psi_angles, residues = ramachandran_plot(pdb_file, chain)
    outliers = classify_outliers(phi_psi_angles)
    outlier_residues = plot_ramachandran(phi_psi_angles, outliers, residues)
    
    # Output the number of outliers and percentage
    num_residues = len(phi_psi_angles)
    num_outliers = len(outliers)
    percentage_outliers = (num_outliers / num_residues) * 100
    
    print(f'Total residues: {num_residues}')
    print(f'Number of outliers: {num_outliers}')
    print(f'Percentage of outliers: {percentage_outliers:.2f}%')
    print('Outliers at residues:', outlier_residues)


In [ ]:

# Run the script with the desired PDB file
file_path = '/Users/adrianahernandezgonzalez/LabNotebook/10-24/states/partialAlphaCaV12HS8HLPlocalrun_b3702_256_512_10/pdb/model_5_ptm_r12_seed0.pdb' #83612
main(pdb_file, chain='A')  # Optionally, specify the chain
